(INTROOPTCODIGO)=

# 5.1 Introducción optimización de código

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

La implementación de los métodos o algoritmos en el contexto de grandes cantidades de datos o *big data* es crítica al ir a la práctica pues de esto depende que nuestra(s) máquina(s) tarde meses, semanas, días u horas para resolver problemas que se presentan en este contexto. En este contexto la [optimización de código o de software](https://en.wikipedia.org/wiki/Program_optimization) nos ayuda a la eficiencia.


## Temas a considerar en un programa de máquina de alto rendimiento

Para tener un alto performance en un programa de máquina, deben considerarse las siguientes preguntas:

* ¿Qué tanto aprovecha mi programa aspectos como ***data reuse*** y ***data locality***? 

La respuesta nos lleva a pensar en el número de instrucciones por ciclo y el número de ciclos que realiza el procesador. Entiéndase un ciclo por los pasos de leer una instrucción, determinar acciones a realizar por tal instrucción y ejecutar las acciones, ver [liga](https://en.wikipedia.org/wiki/Instruction_cycle).

* ¿Cómo es mi ***data layout*** en el almacenamiento? (forma en la que están almacenados o dispuestos los datos)

Dependiendo de la respuesta podemos elegir una arquitectura de computadoras u otra y así también un algoritmo u otro.

* ¿Cuánto ***data movement*** o ***data motion*** realiza mi programa? (flujo de datos entre los distintos niveles de jerarquía de almacenamiento o entre las máquinas en un clúster de máquinas)

La respuesta implica analizar el tráfico de datos entre las **jerarquías de almacenamiento** (o máquinas si estamos en un clúster de máquinas) y potenciales **bottlenecks**.

## Herramientas que tenemos a nuestra disposición para un programa de máquina de alto rendimiento

* Vectorización que pueden realizar los procesadores.

* Perfilamiento de código para lograr la eficiencia deseada. 

* Programación en lenguajes compilados en lugar de intérpretes (o combinando intérpretes con lenguajes compilados)

* Conocimiento de los propósitos con los que fueron diseñados los procesadores para explotar su capacidad. Aquí decidimos si usamos **código secuencial** o **código en paralelo**.

...además necesitamos conocer las diferentes **arquitecturas** que pueden utilizarse para cómputo en paralelo.

* ¿Alguien ya resolvió mi bottleneck?

* Experiencia en el lenguaje de programación seleccionado. 


## Vectorización, BLAS y el uso del caché eficientemente.

El cómputo matricial está construído sobre una jerarquía de operaciones del álgebra lineal:

* Productos punto involucran operaciones escalares de suma y multiplicación (nivel BLAS 1).

* La multiplicación matriz-vector está hecha de productos punto (nivel BLAS 2).

* La multiplicación matriz-matriz utiliza colecciones de productos matriz-vector (nivel BLAS 3).

Las operaciones anteriores se describen en el álgebra lineal con la teoría de espacios vectoriales pero también es posible describirlas en una forma algorítmica. Ambas descripciones se complementan una a la otra.

Ver [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) para nombres que se utilizan para operaciones con escalares, vectores o matrices. Ver [Reference-LAPACK / lapack](https://github.com/Reference-LAPACK/lapack).

### Implementaciones de la API standard de BLAS y LAPACK

```{margin}

Ver [Application Programming Interface: API](https://en.wikipedia.org/wiki/Application_programming_interface) para una explicación de lo que es una API.

```

En [Handle different versions of BLAS and LAPACK](https://wiki.debian.org/DebianScience/LinearAlgebraLibraries) se explica que [BLAS: Basic Linear Algebra Subprograms](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) y [Linear Algebra Package: LAPACK](http://www.netlib.org/lapack/explore-html/dir_fa94b7b114d387a7a8beb2e3e22bf78d.html) además de ser implementaciones, también son API *standard* para operaciones básicas del álgebra lineal. Muchas implementaciones de la API existen. Un ejemplo de implementaciones son las incluidas al instalar R o Python. Otras son las que se pueden instalar vía línea de comando: 

```{margin}

Ver [libblas3](https://packages.debian.org/libblas3) [libblas-dev](https://packages.debian.org/libblas-dev) [liblapack3](https://packages.debian.org/liblapack3) [liblapack-dev](https://packages.debian.org/liblapack-dev).

```

```bash
sudo apt-get install -y libblas3 libblas-dev liblapack3 liblapack-dev
```

en un sistema operativo Ubuntu por ejemplo. 

Sin embargo existen otras implementaciones de la API que están optimizadas para la arquitectura de nuestras máquinas, por ejemplo:

* [OpenBLAS](https://github.com/xianyi/OpenBLAS)

* [Atlas](http://math-atlas.sourceforge.net)



## ¿Qué es el perfilamiento y por qué es necesario?

```{epigraph}

Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered

--  D. Knuth
```

```{epigraph}

We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%". A good programmer ... will be wise to look carefully at the critical code; but only after that code has been identified

--  D. Knuth
```

El perfilamiento de código nos ayuda a encontrar ***bottlenecks*** de nuestro código ya sea en el uso de CPU, RAM, *network bandwidth* u operaciones hacia el disco de *Input*/*Output* (I/O). 

Una mala práctica que es común al iniciar en la programación\* es intentar optimizar el código (y por optimización piénsese en algún caso, por ejemplo mejorar el tiempo de ejecución de un bloque de código) a ciegas, intentando cambiar líneas de código por intuición y no por evidencias o mediciones. Esto aunque puede funcionar en algunas ocasiones no conduce la mayoría de las veces a corregir los problemas de los bottlenecks del código (o bien en programas o ¡sistemas enteros!). La optimización guiada por la intuición conduce a un mayor tiempo en el desarrollo para un pequeño incremento en el performance.

\*Aún teniendo experiencia en programación identificar *bottlenecks* en los códigos es difícil...

Si bien es importante que el código resuelva un problema definido, también es importante perfilarlo. Considérese el caso en el que un código resuelve bien un problema en un día completo, es importante entonces perfilarlo para realizar mediciones (CPU, memoria p.ej.) de los lugares en los que el código gasta la mayor parte de tiempo (en general recursos). 

A largo plazo el perfilamiento de código te dará las decisiones más pragmáticas posibles con el menor esfuerzo total. 

Al perfilar tu código no olvides lo siguiente:

* Medir el tiempo total de tus códigos para decidir si se requiere optimizarlos.
* Perfilar tus códigos para decidir en dónde se iniciará con la optimización. También ayuda definir hipótesis para decidir en qué bloques perfilar primero.
* Escribir *tests* para asegurarse que se resuelve el problema de forma correcta al igual que antes de perfilarlo y optimizarlo.
* Cualquier recurso medible puede ser perfilado (no sólo el uso de CPU p.ej.) 
* Perfilar típicamente añade overhead en la ejecución del código (disminución de tiempo de 10x o 100x es común).

También considera el tiempo que inviertes para optimizar tu código y si vale la pena la inversión de tiempo que realizas en esto pues hay códigos que casi no son utilizados y otros que sí. No pierdas de vista:

<img src="https://imgs.xkcd.com/comics/is_it_worth_the_time.png" heigth="400" width="400">

pues es fácil caer en tratar de remover todos los bottlenecks. Sé una persona práctica, define un tiempo objetivo para tu código y optimiza sólo para llegar a ese objetivo. 

Algunas ***strategies to profile your code successfully*** extraídas de [high performance python](https://www.oreilly.com/library/view/high-performance-python/9781492055013/) para un perfilamiento de código exitoso:

* Disable TurboBoost in the BIOS (a cool CPU may run the same block of code faster than a hot CPU).

* Disable the operating system's ability to override the SpeedStep (you will find this in your BIOS if you're allowed to control it).

* Only use mains power (never battery power) <- a laptop on battery power is likely to more agressively control CPU speed than a laptop on mains power.

* Disable background tools like backups and Dropbox while running experiments.

* Run the experiments many times to obtain a stable measurement.

* Possibly drop to run level 1 (Unix) so that no other tasks are running.

* Reboot and rerun the experiments to double-confirm the results.



### *Unit testing*

Además del perfilamiento del código, el *unit testing* ayuda a **validar** que cada unidad del software trabaje y se desempeñe como fue diseñada (una unidad puede ser una función, programa, procedimiento, método). El *unit testing* es importante pues se debe cuidar que el código genere resultados correctos. Puede realizarse independientemente del perfilamiento y si se ha hecho perfilamiento es muy indispensable que se haga un unit testing.

* ***Unit testing during optimization to maintain correctness:*** ... *after spending a day optimizing her code, having disabled unit tests because they were inconvenient, only to discover that her significant speedup result was due to breaking a part of the algorithm she was improving...*

* *...If you try to performance test code deep inside a larger project without separating it from the larger project, you are likely to witness side effects that will sidetrack your efforts. It is likely to be harder to unit test a larger project when you're making fine-grained changes, and this may further hamper your efforts. Side effects could include other threads and processes impacting CPU and memory usage and network and disk activity, which will skew your results.*

## ¿Por qué compilar a código de máquina?

De las opciones más sencillas que tenemos a nuestra disposición para resolver *bottlenecks* en nuestro programa es hacer que nuestro código haga menos trabajo. 

Compilando nuestro código a código de máquina para que el código en los lenguajes tipo intérpretes ejecuten menos instrucciones es una opción a seguir.

```{margin}

Ver [liga](https://en.wikipedia.org/wiki/Interpreter_(computing)) para revisar lo que es un lenguaje tipo intérprete.

```

### ¿Por qué puede ser lenta la ejecución de un bloque de código en Python (o en algún otro lenguaje tipo intérprete)?

* Verificación de tipo de datos (si son `int`, `double` o `string`).

* Los objetos temporales que se crean por tipo de dato (un objeto tipo `int` en Python tiene asociado un objeto de alto nivel con el que interactuamos pero que causa overhead).

* Las llamadas a funciones de alto nivel (por ejemplo las que ayudan a almacenar al objeto en memoria) 


son tres de las fuentes que hacen a un lenguaje tipo intérprete como `Python` `R` o `Matlab` lento. También otras fuentes son:

* Desde el punto de vista de la memoria de la máquina, el número de referencias a un objeto y las copias entre objetos. 

* No es posible vectorizar un cálculo sin el uso de extensiones (por ejemplo paquetes como `numpy`).

```{admonition} Comentarios

* Hay paquetes que permiten la compilación hacia lenguajes más eficientes como Fortran o C (lenguajes que deben realizar compilación), por ejemplo [cython](https://cython.org/) o [rcpp](http://www.rcpp.org/).

* Escribir directamente en C en un equipo de desarrollo de *software* indudablemente cambiará la velocidad de su trabajo si no conocen tal lenguaje. 

* En la práctica si se tiene un *bottleneck* que no ha podido resolverse con herramientas como el cómputo en paralelo o vectorización, se recomienda utilizar paquetes para regiones pequeñas del código y resolver el *bottleneck* del programa.

```

## Sobre los términos concurrencia, paralelo y distribuido

La distinción entre los términos de paralelo y distribuido es borrosa y en ocasiones es díficil de distinguir. 

### Paralelo

El término **paralelo** típicamente se relaciona con programas cuya ejecución involucra *cores* o nodos que físicamente son cercanos y comparten memoria o están conectados por una red (*network*). 

```{admonition} Observación
:class: tip

El término *core* hoy en día lo usamos como sinónimo de procesador.

```

### Distribuido

Los programas **distribuidos** son ejecutados por nodos o máquinas separadas a  distancia y una de sus características es que no necesariamente fueron iniciados por un nodo central o *master* por lo que su ejecución es independiente de los demás nodos.

### Concurrencia

El término de **concurrencia** se refiere a que las múltiples tareas que debe realizar un programa pueden estar en progreso en cualquier instante. 

```{margin}

Ver [kernel operating system](https://en.wikipedia.org/wiki/Kernel_(operating_system)) para definición del kernel de una máquina.

```

Por ejemplo: cada vez que tu código lee un archivo o escribe a un dispositivo, debe pausar su ejecución para contactar al kernel del sistema operativo, solicitar que se ejecute tal operación, y esperar a que se complete (otra operación que requiere contactar al kernel es alojamiento de memoria). 

Estas operaciones son órdenes de magnitud más lentas que las instrucciones u operaciones ejecutadas en la CPU y el tiempo que el programa espera a que se completen tales operaciones se le llama *I/O wait*. 

La concurrencia nos ayuda a utilizar este tiempo perdido al permitir ejecutar operaciones mientras que una operación I/O se complete. 

Un programa concurrente en lugar de ejecutarse de forma secuencial, esto es, pasar de una línea a otra línea, tiene código escrito para ejecutar distintas líneas conforme sucedan "eventos". Aquí se involucra una forma de programación llamada **asíncrona**, por ejemplo si una operación tipo I/O es solicitada, el programa ejecuta otras funciones mientras espera que se complete la operación I/O.

```{admonition} Comentario

Cambiar de función a función en un programa asíncrono también genera costo pues el kernel debe invertir tiempo en hacer todo el *set up* para ejecutar a la función. La concurrencia funciona bien en programas con alto I/O wait.

```

### ¿Por qué el cómputo en paralelo?

La industria entre los años $2003-2005$ en lugar de fabricar procesadores monolíticos (clásico [Von Neumann](https://en.wikipedia.org/wiki/Von_Neumann_architecture)) que fueran más rápidos y complejos, decidió fabricar múltiples, simples procesadores, *cores*, en un sólo chip para incrementar el poder de procesamiento, disminuir el Von Neumann bottleneck y aumentar el clock speed. 

Esto fue motivado pues desde el año $2002$ el incremento del performance de los procesadores con un sólo CPU fue de un $20\%$ por año vs un $50\%$ por año entre $1986$ y $2002$. Lo anterior se debió a los problemas de la construcción de procesadores monolíticos o de un *core* relacionados con la disipación del calor por un mayor consumo de energía al hacer más pequeños los transistores.

Hoy en día podemos encontrar en nuestros celulares, laptops, computadoras de escritorio y servidores arquitecturas que cuentan con múltiples *cores* o núcleos para procesamiento.

<img src="https://dl.dropboxusercontent.com/s/k11qub01w4nvksi/CPU_multicore.png?dl=0" heigth="500" width="500">


Muchos de los dispositivos anteriores además tienen un(os) procesador(es) gráficos.

<img src="https://dl.dropboxusercontent.com/s/lw9kia12qhwp95r/GPU.png?dl=0" heigth="500" width="500">


En una buena cantidad de aplicaciones tenemos que implementar algoritmos considerando tal disponibilidad de *cores* para reducir el tiempo de procesamiento. Esto conduce a reimplementar o en otros casos a repensar al algoritmo en sí.


Y otro aspecto a tomar en cuenta en esta implementación es la transferencia de datos que existe en la jerarquía de memoria de una máquina

<img src="https://dl.dropboxusercontent.com/s/ahxsnpgp4rjdvw3/jerarquias_de_almacenamiento.png?dl=0" heigth="400" width="400">


```{admonition} Definición 

Los algoritmos que utilizan un sólo *core* para procesamiento les nombramos **secuenciales**, los que utilizan múltiples *cores* son **paralelos**.

```

### Programas cuya ejecución es en paralelo

La decisión de reescribir tu programa secuencial en un programa en paralelo depende mucho de considerar cuatro situaciones:

* Tener el hardware para ejecución en paralelo del programa.

* Comunicarle al programa que está en un hardware para ejecución en paralelo de instrucciones.

* Tener un método que aproveche el hardware paralelo de forma eficiente.

* Tiempo invertido en la reescritura de un código secuencial a uno en paralelo vs ganancias en tiempo de ejecución.

Lo primero es fácilmente alcanzable pues hoy en día tenemos celulares con múltiples *cores* o procesadores. Lo segundo es más dependiente del lenguaje e implementación de éste lenguaje en el que se esté programando. El tercer punto es quizás el más complicado de lograr pues en ocasiones implica repensar el método, disminuir la comunicación lo más posible entre los procesadores, el balanceo de carga o ***load balancing*** debe evaluarse y el perfilamiento o el *debugging* es más difícil en la programación en paralelo que en la forma secuencial. El cuarto punto es esencial para la decisión.

### ¿Cuando es recomendable pensar en ejecutar en paralelo tu programa?

Si tus instrucciones a realizar pueden ser divididas en múltiples *cores* o nodos sin tanto esfuerzo de ingeniería (levantar un clúster de cero es difícil...) o no te lleva mucho tiempo el rediseño de tus métodos para decisiones prácticas (paralelizar el método de despomposición en valores singulares, SVD, es difícil de realizar...) entonces es una opción a considerar. Se recomienda mantener el nivel de paralelización lo más simple posible (aunque no se esté utilizando el 100\% de todos tus *cores*) de modo que el desarrollo de *software* sea rápido.

### Si tengo n procesadores ¿espero un *speedup* de $nx$?

Normalmente **no** se tiene un mejoramiento en la velocidad de $n$ veces ($nx$) (por ejemplo, si tienes una máquina de $8$ *cores* es poco probable que observes un $8x$ *speedup*). 

Las razones de esto tienen que ver con que al paralelizar instrucciones típicamente se incrementa el *overhead* de la comunicación entre los procesos o threads y decrece la memoria RAM disponible que puede ser usada por subprocesos o threads. 

También dentro de las razones se encuentran la [ley de Amdahl](https://en.wikipedia.org/wiki/Amdahl%27s_law) que nos dice que si sólo una parte del código puede ser paralelizado, no importa cuántos cores tengas, en términos totales el código no se ejecutará más rápido en presencia de secciones secuenciales que dominarán el tiempo de ejecución.

### ¿A qué nos referimos al escribir *overhead* en un programa cuya ejecución es en paralelo?

A todo lo que implica ejecutar el programa en paralelo que no está presente en la ejecución en una forma secuencial. Por ejemplo, iniciar procesos implica comunicación con el kernel del sistema operativo y por tanto, tiempo.

### ¿Cuáles enfoques puedo utilizar para escribir programas en paralelo?

Hay $2$ enfoques muy utilizados para escribir programas en paralelo:

* Paralelizar las tareas entre los cores. Su característica principal es la ejecución de instrucciones distintas en los cores. Por ejemplo: al llegar la persona invitada a casa, María le ofrecerá de tomar y Luis le abrirá la puerta.

* Paralelización de los datos entre los cores. Su característica principal es la ejecución de mismas instrucciones en datos que fueron divididos por alguna metodología previa. Por ejemplo: tú repartes la mitad del pastel a las mesas 1,2 y 3, y yo la otra mitad a las mesas 4,5 y 6.


```{admonition} Comentarios

* No son enfoques excluyentes, esto es, podemos encontrar ambos en un mismo programa. La elección de alguno de éstos enfoques depende del problema y del software que será usado para tal enfoque.

* Obsérvese que en el ejemplo de María y Luis necesitan **coordinarse**, **comunicarse** y **sincronizarse** para tener éxito en recibir a la invitada.

* Obsérvese que en el ejemplo de repartir el pastel se requiere un buen **load balancing** pues no queremos que yo le reparta a $5$ mesas y tú le repartas a sólo una!.

```

### ¿A qué nos referimos con el término *embarrassingly parallel problem*?



A los problemas en los que la comunicación entre procesos o threads es cero. Por ejemplo sumar un array `a` con un array `b`.

Y en general si evitamos compartir el estado (pensando a la palabra "estado" como un término más general que sólo comunicación) en un sistema paralelo nos hará la vida más fácil (el *speedup* será bueno, el *debugging* será sencillo, el perfilamiento será más fácil de realizar...).

### ¿Cómo inicio en la programación en paralelo de mi código?

Ian Foster en su libro *Designing and Building Parallel Programs* da una serie de pasos que ayudan a la programación en paralelo:

* *Partitioning. Divide the computation to be performed and the data operated on by the computation into small tasks. The focus here should be on identifying tasks that can be executed in parallel.*

* *Communication. Determine what communication needs to be carried out among the tasks identified in the previous step.*

* *Agglomeration or aggregation. Combine tasks and communications identified in the first step into larger tasks. For example, if task A must be executed before task B can be executed, it may make sense to aggregate them into a single composite task.*

* *Mapping. Assign the composite tasks identified in the previous step to processes/threads. This should be done so that communication is minimized, and each process/thread gets roughly the same amount of work.*



```{admonition} Comentario

En ocasiones es mejor dejar las tareas simples y redundantes que complicarlas y no redundantes. Por ejemplo, si en el programa en paralelo varios procesadores o threads hacen tarea redundante pero me evitan la comunicación, prefiero este programa a uno en el que haga  trabajo no redundate y muy específico a cada procesador o thread pero que la comunicación a realizar sea muy complicada o compleja.

```

### Ejemplo en la regla del rectángulo compuesta en una máquina *multicore* para CPU

1. *Partitioning*: la tarea a realizar es el cálculo de un área de un rectángulo para un subintervalo.

<img src="https://dl.dropboxusercontent.com/s/5nqciu6ca5xzdh9/parallel_processing_Rcf_1.png?dl=0" heigth="300" width="400">

2. *Communication* y *mapping*: los subintervalos deben repartirse entre los *cores* y se debe comunicar esta repartición por algún medio (por ejemplo con variables en memoria).

3. *Aggregation*: un *core* puede calcular más de un área de un rectángulo si recibe más de un subintervalo. 


<img src="https://dl.dropboxusercontent.com/s/lpcwd9mejb90rq3/parallel_processing_Rcf_2.png?dl=0" heigth="200" width="300">


4. *Communication* y *mapping*: el área de los rectángulos calculados por cada procesador deben sumarse para calcular la aproximación a la integral.

<img src="https://dl.dropboxusercontent.com/s/mfo5rfzjnonn8lq/parallel_processing_Rcf_3.png?dl=0" heigth="400" width="500">



### ¿Software?


Depende si lo que deseamos usar son *cores* en una CPU tenemos:

* [Dask](https://docs.dask.org/en/latest/), [multiprocessing](https://docs.python.org/3.1/library/multiprocessing.html), [parallel](https://www.rdocumentation.org/packages/parallel/versions/3.6.2),  [openMP](http://www.openmp.org/), [Thrust](https://thrust.github.io/) ...

Si deseamos usar *cores* en una GPU tenemos:

* [CUDA C](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html), [CuPy](https://docs.cupy.dev/en/stable/), [PyCUDA](https://documen.tician.de/pycuda/), [gputools](https://www.rdocumentation.org/packages/gputools/versions/1.1), [Thrust](https://thrust.github.io/) ....


```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Preguntas de comprehensión:**


1)Menciona la ley de Amdahl.

2)¿Qué significan los términos concurrencia, paralelo, distribuido?

3)¿Cuáles son los enfoques que se utilizan para escribir programas en paralelo?

4)Define a cuál enfoque corresponde (de acuerdo a la pregunta 13) cada uno de los siguientes incisos:

a) Supón que tienes 2 cores y un arreglo de tamaño 100

if(rango_core módulo 2 == 0 )
    operar en los elementos 50 a 99
else
    operar en los elementos 0 a 49
        
Donde módulo es una operación que nos devuelve el residuo al dividir un número entre otro.

b) Tenemos tres trabajadores: Aurora, Pedro, Daniel
    
if(mi_nombre es Pedro)
    lavo el baño
else
    voy de compras
            
5)En el cómputo en paralelo debemos realizar coordinación entre procesos o threads y considerar el load balancing. Menciona tipos de coordinación que existen y ¿a qué se refiere el load balancing?

6)¿Cuáles son los pasos a seguir, que de acuerdo a Ian Foster, se puede seguir para el diseño de programas en paralelo?



**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

3. P. Pacheco, An Introduction to Parallel Programming, Morgan Kaufmann, 2011.

4. https://xkcd.com/
